In [4]:
import dotenv
import sagemaker
import boto3
import os
from datasets import load_dataset

In [5]:
dotenv.load_dotenv()

True

In [18]:
boto3.setup_default_session(profile_name="ai-sandbox-sso")

In [19]:
boto3.client("sts").get_caller_identity()

{'UserId': 'AROAX4Q5STILGJLLHY6I6:nasoungadoy',
 'Account': '542301657622',
 'Arn': 'arn:aws:sts::542301657622:assumed-role/AWSReservedSSO_AdministratorAccess_bfd3f71331b64628/nasoungadoy',
 'ResponseMetadata': {'RequestId': '86ea6e0b-1404-41b2-9acd-15993919585f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '86ea6e0b-1404-41b2-9acd-15993919585f',
   'content-type': 'text/xml',
   'content-length': '480',
   'date': 'Fri, 13 Oct 2023 14:57:06 GMT'},
  'RetryAttempts': 0}}

In [20]:
role_name = os.environ[ "SAGEMAKER_ROLE_NAME" ]

In [21]:
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

# try:
#     role = sagemaker.get_execution_role()
# except ValueError:
iam = boto3.client("iam")
role = iam.get_role(RoleName=role_name)["Role"]["Arn"]

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/nsoungadoy2/Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/nsoungadoy2/Library/Application Support/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::542301657622:role/service-role/AmazonSageMaker-ExecutionRole-20231002T234337
sagemaker bucket: sagemaker-us-east-1-542301657622
sagemaker session region: us-east-1


In [22]:
input_dataset_s3_path = f's3://{sess.default_bucket()}/processed/mbay/t5/'

In [23]:
input_dataset_s3_path

's3://sagemaker-us-east-1-542301657622/processed/mbay/t5/'

In [ ]:
TOKENIZED_DATASET_PATH = "../../datasets/mbay-translations-tokenized/"
dataset = load_dataset(TOKENIZED_DATASET_PATH)

In [ ]:
from datasets import load_dataset

print("uploaded data to:")
# Load dataset from disk and upload to S3
input_dataset_s3_path = f's3://{sess.default_bucket()}/processed/mbay/t5/'
dataset.save_to_disk(input_dataset_s3_path)
print(f"training dataset to: {input_dataset_s3_path}")

sample_dataset_s3_path = f's3://{sess.default_bucket()}/processed/mbay-samples/t5/'
sampled_dataset = dataset.shuffle().select(range(int(len(dataset)*0.1)))
sampled_dataset.save_to_disk(sample_dataset_s3_path)
print(f"sample dataset to: {sample_dataset_s3_path}")

In [3]:
from datasets import DatasetDict
DatasetDict.select?

Object `DatasetDict.select` not found.


In [ ]:
import time
from mbay_nmt.training.core import training_job_name
from sagemaker.huggingface import HuggingFace
from huggingface_hub import HfFolder

use_spot_instances = True
model_id = "t5-base"

# define Training Job Name
job_name = training_job_name(model_id)

# hyperparameters, which are passed into the training job
hyperparameters = {
    "model_id": model_id,  # pre-trained model
    "dataset_path": "/opt/ml/input/data",  # path where sagemaker will save training dataset
    "epochs": 3,  # number of training epochs
    "per_device_train_batch_size": 2,  # batch size for training
    "lr": 2e-4,  # learning rate used during training
    "hf_token": HfFolder.get_token(),  # huggingface token to access llama 2
    "merge_weights": True,  # wether to merge LoRA into the model (needs more memory) // we only have access to g5.2xlarge. This is not enough memory
}


# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point="run.py",  # train script
    source_dir="../../run_fine_tune_t5",  # directory which includes all the files needed for training
    instance_type="ml.g5.2xlarge",  # instances type used for the training job
    instance_count=1,  # the number of instances used for training
    base_job_name=job_name,  # the name of the training job
    role=role,  # Iam role used in training job to access AWS ressources, e.g. S3
    volume_size=100,  # the size of the EBS volume in GB
    transformers_version="4.28",  # the transformers version used in the training job
    pytorch_version="2.0",  # the pytorch_version version used in the training job
    py_version="py310",  # the python version used in the training job
    hyperparameters=hyperparameters,  # the hyperparameters passed to the training job
    # use_spot_instances   =  use_spot_instances, # wether to use spot instances or not
    environment={
        "HUGGINGFACE_HUB_CACHE": "/tmp/.cache"
    },  # set env variable to cache models in /tmp
)

In [ ]:
# define a data input dictonary with our uploaded s3 uris
# data = {"training": training_input_path}

# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(input_dataset_s3_path)